This script gets a map representing the main effect in the MCSE study, extracts the peak coordinates from it
and then fetches anatomical labels for these regions


In [ ]:
# Let us load the image obtained from the previous group analysis.
# we are interested in the low-high salience contrast only
import os
from nilearn.image import load_img
write_dir = 'results'
key = 'low-high salience'
filename = os.path.join(write_dir, f'group_{key}_z_score.nii.gz')
z_map = load_img(filename)

In [ ]:
from nilearn.reporting import get_clusters_table
# we need to threshold the maps. We keep
threshold = 3.06

table = get_clusters_table(
    z_map, stat_threshold=threshold, cluster_threshold=20
)
table

We see 37 main peaks organized in 14 clusters.
Now let us try to make sense of these clusters by observing then in the Julich atlas space.

To fetch the Julich-Brain from EBRAINS we rely on the `siibra` library. So we first need to import it (or install and then import if not already installed)

In [ ]:
try:
    import siibra
except:
    !pip install siibra
    import siibra
print(siibra.__version__)
assert siibra.__version__ >= "0.4a33"

Now we can fetch the Julich-Brain parcellation as a labelled map in MNI space 

In [ ]:
julich_brain = siibra.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_9
print(julich_brain.description)

# the original maximum probability map object with split hemispheres
julich_mpm = julich_brain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.LABELLED
)

# single-volume compressed version
julich_mpm_compressed = julich_mpm.compress()
parcellation_map_niimg = julich_mpm_compressed.fetch()

In [ ]:
from nilearn.image import resample_to_img

parcellation_map_niimg = resample_to_img(parcellation_map_niimg, z_map, interpolation='nearest')

You can access the regions defined in the Julich map with the `regions` attribute

In [ ]:
julich_mpm_compressed.regions

With Nilearn we can visualize the thresholded statistical map on the Julich parcellation map

In [ ]:
from nilearn import plotting

plotting.view_img(z_map, bg_img=parcellation_map_niimg, threshold=threshold)

Get labels from atlas using peak coordinates from cluster table

In [ ]:
import numpy as np 

labels = []
for x, y, z in zip(table["X"], table["Y"], table["Z"]):
    coord = np.array([x, y, z]).astype("int")
    labels.append(parcellation_map_niimg.get_fdata()[coord])

Map labels back to atlas